## [Multinomial Logistic Regression with Tensorflow from Youtube](https://www.youtube.com/watch?v=2JiXktBn_2M)

In [1]:
import numpy as np
import pandas as pd

#image tools
import matplotlib.pyplot as plt
import image

#filesystem tools
import os
from glob import glob

import tensorflow as tf

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
def accuracy(prediction,labels):
    return (100.0 * (np.sum(np.equal(np.argmax(prediction, 1), np.argmax(labels, 1))))
           /prediction.shape[0])

In [3]:
graph = tf.Graph()
with graph.as_default():
    
    # Variables
    batch_size = 128
    beta = .01
    image_size = 28
    num_labels = 10
    learning_rate = 0.01
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(mnist.validation.images)
    tf_test_dataset = tf.constant(mnist.test.images)
    
    # Weights and biases for output/logit layer
    w_logit = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels])) 
    b_logit = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):
        """
        Assembles the NN
        """
        return tf.matmul(data, w_logit) + b_logit
                          
    # Training Computations
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits,labels= tf_train_labels))
    regularized_loss = tf.nn.l2_loss(w_logit)
    total_loss = loss + beta + regularized_loss
    
    # Optimizer:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    
    # Predictions for the training, validation and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [5]:
num_steps = 501

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        #Generate a minibatch.
        batch_data, batch_labels = mnist.train.next_batch(batch_size)
    
        #Dictionary to feed
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict = feed_dict) 
    
        if (step % 25 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), mnist.validation.labels))
            
    W_val = session.run(w_logit)
    b_val = session.run(b_logit)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), mnist.test.labels))
    print('W:',W_val)
    print('B:',b_val)

Initialized
Minibatch loss at step 0: 11.867926
Minibatch accuracy: 12.5%
Validation accuracy: 8.9%
Minibatch loss at step 25: 8.110176
Minibatch accuracy: 9.4%
Validation accuracy: 9.1%
Minibatch loss at step 50: 6.076201
Minibatch accuracy: 14.1%
Validation accuracy: 9.7%
Minibatch loss at step 75: 4.282178
Minibatch accuracy: 14.1%
Validation accuracy: 11.6%
Minibatch loss at step 100: 3.416584
Minibatch accuracy: 10.2%
Validation accuracy: 14.3%
Minibatch loss at step 125: 2.860052
Minibatch accuracy: 18.8%
Validation accuracy: 18.8%
Minibatch loss at step 150: 2.267735
Minibatch accuracy: 25.0%
Validation accuracy: 25.2%
Minibatch loss at step 175: 1.965567
Minibatch accuracy: 30.5%
Validation accuracy: 31.7%
Minibatch loss at step 200: 1.808925
Minibatch accuracy: 41.4%
Validation accuracy: 39.1%
Minibatch loss at step 225: 1.710536
Minibatch accuracy: 53.9%
Validation accuracy: 48.1%
Minibatch loss at step 250: 1.747340
Minibatch accuracy: 50.8%
Validation accuracy: 56.7%
Miniba

## [Multinomial Perceptrons in TensorFlow](https://www.youtube.com/watch?v=fQ8q8LTMzwo)

In [6]:
graph = tf.Graph()
with graph.as_default():
    
    # Variables
    batch_size = 128
    beta = .01
    image_size = 28
    num_labels = 10
    learning_rate = 0.01
    
    #new
    h1_size = 512 #number of neurons in hidden layer
    keep_prob = 0.5 #probability of dropout
    
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(mnist.validation.images)
    tf_test_dataset = tf.constant(mnist.test.images)
    
    # Weights and biases for hidden layer 1
    w_h1 = tf.Variable(tf.truncated_normal([image_size*image_size, h1_size])) 
    b_h1 = tf.Variable(tf.zeros([h1_size]))
    
    # Weights and biases for output/logit layer
    w_logit = tf.Variable(tf.truncated_normal([h1_size, num_labels])) 
    b_logit = tf.Variable(tf.zeros([num_labels]))
    
    #keep probability for dropout. This is a variable so it can be turned on for train and off for predict
    keep_prob = tf.placeholder("float")
    
    def model(data):
        """
        Assembles the NN

        """
        h1 = tf.nn.relu(tf.matmul(data, w_h1) + b_h1)
        h1_drop_out = tf.nn.dropout(h1,keep_prob)
        return tf.matmul(h1_drop_out, w_logit) + b_logit
                          
    # Training Computations
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits,labels= tf_train_labels))
    regularized_loss = tf.nn.l2_loss(w_logit)
    total_loss = loss + beta + regularized_loss
    
    # Optimizer:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    
    # Predictions for the training, validation and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 301

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        #Generate a minibatch.
        batch_data, batch_labels = mnist.train.next_batch(batch_size)
    
        #Dictionary to feed
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels: batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict = feed_dict) 
    
        if (step % 25 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={keep_prob:1.0}), 
                                                           mnist.validation.labels.astype(float)))
            #feed_dict={keep_prob:1.0}
    W_val = session.run(w_logit)
    b_val = session.run(b_logit)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob:1.0}), mnist.test.labels))
    print('W:',W_val)
    print('B:',b_val)
    

Initialized
Minibatch loss at step 0: 301.676483
Minibatch accuracy: 9.4%
Validation accuracy: 5.8%
Minibatch loss at step 25: 118.703156
Minibatch accuracy: 17.2%
Validation accuracy: 29.2%
Minibatch loss at step 50: 65.175095
Minibatch accuracy: 30.5%
Validation accuracy: 52.1%
Minibatch loss at step 75: 36.946068
Minibatch accuracy: 39.1%
Validation accuracy: 64.2%
Minibatch loss at step 100: 21.793938
Minibatch accuracy: 47.7%
Validation accuracy: 71.7%
Minibatch loss at step 125: 13.897432
Minibatch accuracy: 50.0%
Validation accuracy: 76.1%
Minibatch loss at step 150: 7.452451
Minibatch accuracy: 57.8%
Validation accuracy: 79.2%
Minibatch loss at step 175: 8.250422
Minibatch accuracy: 57.0%
Validation accuracy: 81.7%
Minibatch loss at step 200: 4.189260
Minibatch accuracy: 57.8%
Validation accuracy: 83.9%
Minibatch loss at step 225: 3.765678
Minibatch accuracy: 64.1%
Validation accuracy: 84.0%
Minibatch loss at step 250: 2.381036
Minibatch accuracy: 68.8%
Validation accuracy: 84.

## [Convolutional Neural Networks in Tensorflow](https://www.youtube.com/watch?v=lTFOw8-P02Y)

In [4]:
from PIL import Image

In [10]:
graph = tf.Graph()
with graph.as_default():
    
    # Variables
    batch_size = 128
    beta = .01
    image_size = 28
    num_labels = 10
    learning_rate = 0.01
    
    #second part
    h1_size = 1024 #number of neurons in hidden layer
    keep_prob = 0.5 #probability of dropout
    
    #new 
    patch_size = 5
    num_channels = 1 #grayscale
    depth = 16
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(0.1, shape = shape)
        return tf.Variable(initial)
    
    def conv2d(x,W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size,image_size,num_channels ))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(mnist.validation.images.reshape(-1,28,28,1))
    tf_test_dataset = tf.constant(mnist.test.images.reshape(-1,28,28,1))
    
    # Weights and biases 
    w_conv1 = weight_variable([5,5,1,32])
    b_conv1 = bias_variable([32])
    w_conv2 = weight_variable([5,5,32,64])
    b_conv2 = bias_variable([64])
    w_fc1 = weight_variable([7 * 7  * 64, 1024])
    b_fc1 = bias_variable([1024])
    w_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])

    #keep probability for dropout. This is a variable so it can be turned on for train and off for predict
    keep_prob = tf.placeholder(tf.float32)
    
    def model(data):
        """
        Assembles the NN

        """
        h_conv1 = tf.nn.relu(conv2d(data, w_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)
        h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
        return tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)
                          
    # Training Computations
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits,labels= tf_train_labels))
    
    # Optimizer:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions for the training, validation and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        #Generate a minibatch.
        batch_data, batch_labels = mnist.train.next_batch(batch_size)
        
        #reshape
        batch_data = batch_data.reshape(-1, 28, 28, 1)
    
        #Dictionary to feed
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels: batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict = feed_dict) 
    
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={keep_prob:1.0}), 
                                                           mnist.validation.labels.astype(float)))
            #feed_dict={keep_prob:1.0}
    W_val = session.run(w_fc2)
    b_val = session.run(b_fc2)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob:1.0}), mnist.test.labels))
    print('W:',W_val)
    print('B:',b_val)

Initialized
Minibatch loss at step 0: 2.344798
Minibatch accuracy: 9.4%
Validation accuracy: 16.8%
Minibatch loss at step 100: 2.274041
Minibatch accuracy: 15.6%
Validation accuracy: 28.9%
Minibatch loss at step 200: 2.168340
Minibatch accuracy: 30.5%
Validation accuracy: 39.5%
Minibatch loss at step 300: 2.072723
Minibatch accuracy: 37.5%
Validation accuracy: 51.0%
Minibatch loss at step 400: 1.953208
Minibatch accuracy: 50.8%
Validation accuracy: 59.9%
Minibatch loss at step 500: 1.861571
Minibatch accuracy: 60.2%
Validation accuracy: 63.6%
Minibatch loss at step 600: 1.931832
Minibatch accuracy: 53.1%
Validation accuracy: 64.7%
Minibatch loss at step 700: 1.909201
Minibatch accuracy: 54.7%
Validation accuracy: 65.0%
Minibatch loss at step 800: 1.812298
Minibatch accuracy: 65.6%
Validation accuracy: 65.6%
Minibatch loss at step 900: 1.872097
Minibatch accuracy: 59.4%
Validation accuracy: 68.9%
Minibatch loss at step 1000: 1.763211
Minibatch accuracy: 71.1%
Validation accuracy: 80.0%
